In [1]:
import pandas as pd
import numpy as np
sys.path.insert(0, '../src')
import prework as pwk
import string
from time import time

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cross_validation import train_test_split, cross_val_score
from sklearn.feature_extraction.text import TfidfTransformer, CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import LinearSVC
from sklearn.grid_search import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.metrics import recall_score, accuracy_score, precision_score, roc_auc_score
from nltk.stem.snowball import SnowballStemmer
from nltk.stem.wordnet import WordNetLemmatizer
from collections import Counter

import matplotlib.pyplot as plt
%matplotlib inline

In [3]:
df_0 = pd.read_pickle('../data/pickles/20180117_part_one.pkl')

**After gridsearch, window_size=2, hidden_nodes=100 turns out to be the best choice**

## Transform feature 'essay_content'

In [48]:
train_data, test_data = train_test_split(df_0, train_size=0.8)
col = 'essay_content'
train_docs = pwk.tag_docs(train_data, col)
test_docs = pwk.tag_docs(test_data, col)
model = pwk.train_doc2vec_model(train_docs, 2, 100)

y_train, X_train = pwk.vec_for_learning(model, train_docs)
y_test, X_test = pwk.vec_for_learning(model, test_docs)

In [49]:
','.join(train_docs.values[0].words)#

u'agree,because,it,teaches,students,to,take,on,more,responsibility,gives,them,some,extra,income,and,allows,them,to,get,some,very,real,work,experience,it,is,important,that,students,learn,that,they,have,to,earn,sooner,or,later,and,think,that,having,part,time,job,gives,them,the,perfect,introduction,as,to,what,life,will,really,be,like,when,they,enter,their,new,jobs,on,full,time,basis,if,were,in,college,would,be,getting,as,many,part,time,jobs,that,could,squeeze,in,because,want,to,have,many,different,experiences,so,when,and,if,travel,have,more,than,one,set,of,skills,that,can,fall,back,on,if,need,to,get,some,type,of,job,while,away,guess,it,can,be,easy,though,because,with,all,the,study,and,all,and,loads,of,other,students,also,looking,for,part,time,jobs,it,must,get,frustrating,for,them,sometimes,then,again,though,if,they,experience,lot,of,problems,at,college,then,they,get,better,at,dealing,with,them,which,have,got,to,help,them,later,in,life,mean,because,they,have,faced,some,challenges,then,they

In [50]:
# <label>_<doc_id>
','.join(train_docs.values[0].tags)#

'ENS_1127'

In [51]:
X_train[0]

array([  3.80351092e-03,  -2.26163208e-01,  -2.40365833e-01,
        -7.46642053e-01,  -8.73783529e-02,   7.55064338e-02,
        -6.81282207e-02,  -5.81557527e-02,   1.72942907e-01,
         6.89861119e-01,  -1.24780133e-01,  -4.67391282e-01,
        -6.81670129e-01,   3.07710399e-03,  -5.13115644e-01,
         3.06478679e-01,   1.60836563e-01,   1.48783773e-01,
         2.45504789e-02,  -1.33079425e-01,  -1.83911055e-01,
        -8.24804306e-01,  -1.04266211e-01,   1.47078067e-01,
         1.66241392e-01,  -1.55283630e-01,   2.25255966e-01,
        -2.41942659e-01,  -1.82405040e-01,   1.06709227e-01,
        -3.62052247e-02,  -4.31679815e-01,   1.35203779e-01,
        -8.36766735e-02,   2.96481490e-01,  -3.69180858e-01,
         4.46793199e-01,   9.09058377e-04,   3.23891312e-01,
         3.47610712e-01,   1.63281243e-02,  -3.27956527e-01,
        -8.94003212e-01,  -3.66845019e-02,  -3.19157362e-01,
        -3.85363013e-01,  -8.09012949e-01,  -1.52627807e-02,
         3.10770750e-01,

In [52]:
y_train[0]

'ENS'

**Get a rough idea about how this vector is performing for classification**

In [60]:
train_data, test_data = train_test_split(df_0, train_size=0.8)
col = 'essay_content'

clf = LogisticRegression()
print pwk.k_fold_doc2vec_clf(train_data, col, 2, 100, clf)

[0.81770833333333337, 0.82465277777777779, 0.82118055555555558, 0.78645833333333337, 0.828125]


## Transform feature 'DT_insent_arch_ngram'

In [61]:
n = 4
df_0['DT_insent_arch_ngram'] = df_0['DT_archs'].apply(lambda x: pwk.loop_body(x, n))

In [62]:
train_data, test_data = train_test_split(df_0, train_size=0.8)
col = 'DT_insent_arch_ngram'
# train_docs = pwk.tag_docs(train_data, col, literal=False)
# test_docs = pwk.tag_docs(test_data, col, literal=False)
# model = pwk.train_doc2vec_model(train_docs, 1, 100)

# y_train, X_train = pwk.vec_for_learning(model, train_docs)
# y_test, X_test = pwk.vec_for_learning(model, test_docs)

In [63]:
train_data, test_data = train_test_split(df_0, train_size=0.8)
col = 'DT_insent_arch_ngram'

clf = LogisticRegression()
print pwk.k_fold_doc2vec_clf(train_data, col, 1, 100, clf, literal=False)

[0.36805555555555558, 0.35069444444444442, 0.3576388888888889, 0.34722222222222221, 0.37152777777777779]


**Turns out 'essay_content' is the best-performing column for extracting the vector to classify essays.**